In [3]:
import os
import json
import pandas as pd
import traceback

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [8]:
from dotenv import load_dotenv
load_dotenv()
KEY=os.getenv("GEMINI_API_KEY")

In [10]:
llm=ChatGoogleGenerativeAI(google_api_key=KEY,model="gemini-2.0-flash-exp",temperature=0.5)

In [11]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AA2E3B04C0>, default_metadata=(), model_kwargs={})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI   # Gemini LLM

from langchain.prompts import PromptTemplate                 # For custom prompts
from langchain.chains import LLMChain, SequentialChain       # For chaining
import PyPDF2                                                # For PDF reading


In [14]:
RESPONSE_JSON={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "5": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [15]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}


"""

In [16]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [28]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)
#quiz_chain= quiz_generation_prompt | llm

In [24]:
TEMPLATE2=""" 
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at per with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [25]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [ ]:
#review_chain= quiz_evaluation_prompt | llm
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [31]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], 
                                        input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [32]:
file_path=r'C:\Users\Anitha\mcqgen\data.txt'


In [33]:
file_path

'C:\\Users\\Anitha\\mcqgen\\data.txt'

In [34]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [36]:
print(TEXT)

A quantum computer is a (real or theoretical) computer that uses quantum mechanical phenomena in an essential way: a quantum computer exploits superposed and entangled states and the (non-deterministic) outcomes of quantum measurements as features of its computation. Ordinary ("classical") computers operate, by contrast, using deterministic rules. Any classical computer can, in principle, be replicated using a (classical) mechanical device such as a Turing machine, with at most a constant-factor slowdown in time—unlike quantum computers, which are believed to require exponentially more resources to simulate classically. It is widely believed that a scalable quantum computer could perform some calculations exponentially faster than any classical computer. Theoretically, a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing physical simulations. However, current hardware implementations of quantum computation are largely experimen

In [37]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
NUMBER=5
SUBJECT="quantum computing"
TONE="simple"

In [39]:
response=generate_evaluate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)

C:\Users\Anitha\AppData\Local\Temp\ipykernel_3476\1480018621.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:A quantum computer is a (real or theoretical) computer that uses quantum mechanical phenomena in an essential way: a quantum computer exploits superposed and entangled states and the (non-deterministic) outcomes of quantum measurements as features of its computation. Ordinary ("classical") computers operate, by contrast, using deterministic rules. Any classical computer can, in principle, be replicated using a (classical) mechanical device such as a Turing machine, with at most a constant-factor slowdown in time—unlike quantum computers, which are believed to require exponentially more resources to simulate classically. It is widely believed that a scalable quantum computer could perform some calculations exponentially faster than any classical computer. Theoretically, a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing 

In [40]:
response

{'text': 'A quantum computer is a (real or theoretical) computer that uses quantum mechanical phenomena in an essential way: a quantum computer exploits superposed and entangled states and the (non-deterministic) outcomes of quantum measurements as features of its computation. Ordinary ("classical") computers operate, by contrast, using deterministic rules. Any classical computer can, in principle, be replicated using a (classical) mechanical device such as a Turing machine, with at most a constant-factor slowdown in time—unlike quantum computers, which are believed to require exponentially more resources to simulate classically. It is widely believed that a scalable quantum computer could perform some calculations exponentially faster than any classical computer. Theoretically, a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing physical simulations. However, current hardware implementations of quantum computation are largely

In [52]:
import re
raw=response.get("quiz")
quiz= re.sub(r"```json|```", "", raw).strip()


In [54]:
quiz=json.loads(quiz)

In [55]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
            ]
        )
    correct=value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options,"Correct":correct})


In [56]:
quiz_table_data

[{'MCQ': 'What is a key difference between quantum computers and classical computers, according to the text?',
  'Choices': 'a:Quantum computers use deterministic rules, while classical computers use quantum mechanical phenomena. | b:Classical computers can be simulated by Turing machines with a constant-factor slowdown, while quantum computers are believed to require exponentially more resources to simulate classically. | c:Quantum computers are always faster than classical computers for all types of calculations. | d:Classical computers use qubits, while quantum computers use bits.',
  'Correct': 'b'},
 {'MCQ': 'What is a qubit and how does it differ from a classical bit?',
  'Choices': 'a:A qubit is the same as a classical bit, only smaller. | b:A qubit can only be in one of two states, like a classical bit. | c:A qubit can exist in a superposition of its two basis states, unlike a classical bit. | d:A qubit is used in classical computers, while a bit is used in quantum computers.',

In [58]:
quiz=pd.DataFrame(quiz_table_data)

In [59]:
quiz.to_csv("QuantumComputing.csv",index=False)